<a href="https://colab.research.google.com/github/Slimlav/Netology/blob/main/%D0%93%D0%B8%D0%B1%D1%80%D0%B8%D0%B4%D0%BD%D1%8B%D0%B5_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k

In [ ]:
movies=pd.read_csv('movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
ratings=pd.read_csv('ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
df=pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
df

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5
...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),184,4.0
100832,193583,No Game No Life: Zero (2017),184,3.5
100833,193585,Flint (2017),184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5


In [ ]:
df_pivot=df.pivot_table(index = 'userId', values = 'rating',columns='movieId')

In [ ]:
pivot_train_np=df_pivot.to_numpy()
df_ds=pivot_train_np[:-1,1:-1].astype('int')
sData=csr_matrix(df_ds)

In [ ]:
X_train_pivot, X_test_pivot=cross_validation.random_train_test_split(sData, test_percentage=0.2, random_state=None)

In [ ]:
modelFM = LightFM(learning_rate=0.05, loss='warp')
modelFM.fit_partial(X_train_pivot, epochs=30)

In [ ]:
train_precision = precision_at_k(modelFM, X_train_pivot, k=10).mean()
test_precision = precision_at_k(modelFM, X_test_pivot, k=10).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.83, test 0.17.


In [ ]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = X_train_pivot.shape
    for user_id in user_ids:
        known_positives = data['title'][X_train_pivot.tocsr()                                    
                          [user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['title'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:1]:
            print("        %s" % x)
        
        print("     Recommended:")
        
        for x in top_items[:10]:
            print("        %s" % x)
sample_recommendation(modelFM, df, [1])

User 1
     Known positives:
        Toy Story (1995)
     Recommended:
        Star Wars: Episode IV - A New Hope (1977)
        Indian in the Cupboard, The (1995)
        Mary Shelley's Frankenstein (Frankenstein) (1994)
        Batman Forever (1995)
        Apollo 13 (1995)
        Clueless (1995)
        Interview with the Vampire: The Vampire Chronicles (1994)
        Léon: The Professional (a.k.a. The Professional) (Léon) (1994)
        Things to Do in Denver When You're Dead (1995)
        Taxi Driver (1976)
